In [1]:
from Bio import Seq
from collections import defaultdict
import pandas as pd



In [28]:
def generate_words(chars, length):
    if length == 0:
        yield ''
    else:
        for c in chars:
            for suffix in generate_words(chars, length - 1):
                yield c + suffix
                
def get_features():
    nucleotides = generate_words('ACGT', 4)
    seen = set()
    features = []
    nucleotide2feature = dict()
    for n in nucleotides:
        if n in seen or Seq.reverse_complement(n) in seen:
            continue
        nucleotide2feature[n] = n 
        nucleotide2feature[Seq.reverse_complement(n)] = n
        seen.add(n)
        features.append(n)
    return features, nucleotide2feature
        
features, mapping = get_features()

In [18]:
def read_training_data(filename):
    data = []
    with open(filename, 'r') as f:
        lines = f.readlines()
        for line in lines:
            if line.startswith('>'):
                continue
            data.append(line.upper().strip())
    return data

positive_data = read_training_data('./data/vista1500')
negative_data = read_training_data('./data/randoms1500')

In [29]:
def convert_line_to_features(line, label):
    dd = dict()
    for feature in features:
        dd[feature] = 0
    total = len(line) - 4
    for i in range(total):
        dd[mapping[line[i:i+4]]] += 1 / total
    dd['label'] = label
    return dd
    
def convert_data_to_features(data, label):
    data_obj = []
    for line in data:
        data_obj.append(convert_line_to_features(line, label))
    return data_obj

training_data = convert_data_to_features(positive_data, 0) + convert_data_to_features(negative_data, 1) 



In [33]:
len(training_data)
training_df = pd.DataFrame(training_data)

In [32]:
from sklearn.ensemble import RandomForestClassifier

In [41]:
clf = RandomForestClassifier(n_estimators=10)
training_df.values
training_df['label']
clf.fit(training_df, training_df['label'])
# clf = clf.fit(X, Y)

TypeError: Singleton array array('label', dtype='<U5') cannot be considered a valid collection.